<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2024-10-23 21:05:40--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1 [following]
--2024-10-23 21:05:41--  https://www.dropbox.com/scl/fi/3x5umw93vtxvmp037wczv/mieszkania.csv?rlkey=dmvzaueu361g7s2w6ui6m9ryb&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4ab266427867c286ef8956b6ff.dl.dropboxusercontent.com/cd/0/inline/CdC8X2Lp38TMlHsd5o5AAKa7DoaWVZVlYCahsjPcU1ZZOraBDs25RiOsBynwrVA6vA-ELWpzi3LMlr-7dMSmVQcF3-y9IESNVZsuQiO1LsouzFgI0_WGDbiX1z1Rxr5ETEo/file?dl=1# [following]
--2024-10-23 21:05:41--  https://uc4ab266427867c286ef8956b6ff.dl.dropboxusercontent.com/cd/0/inl

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`).

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [3]:
import numpy as np

def msle(ys, ps):
    return np.mean((np.log1p(ys) - np.log1p(ps)) ** 2)

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [4]:
###################################################
# TODO: Compute msle for outputing the mean price #
###################################################
import pandas as pd

mieszkania = pd.read_csv('mieszkania.csv')
prizes = mieszkania['cena'].to_numpy()
msle(prizes, np.full(len(prizes), np.mean(prizes)))

0.3915253538257009

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [5]:
#############################################
# TODO: Find this constant and compute msle #
#############################################
# zi = log(1 + yi)
# 1/n sum (zi - log(1 + c))^2
# a = log(1 + c) -> 1/n sum (zi - a)^2
# we know that a := mean(...) minimizes, so
# a = mean(zi) -> c = exp(mean(zi)) - 1
z = np.log1p(prizes)
c = np.exp(np.mean(z)) - 1
msle(prizes, np.full(len(prizes), c))


0.36488961221335886

In [6]:
mieszkania.head()

,m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
0,104,mokotowo,2,2,1940,1,780094
1,43,ochotowo,1,1,1970,1,346912
2,128,grodziskowo,3,2,1916,1,523466
3,112,mokotowo,3,2,1920,1,830965
4,149,mokotowo,3,3,1977,0,1090479


from matplotlib import pyplot as plt
_df_0['m2'].plot(kind='hist', bins=20, title='m2')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['ilość_sypialni'].plot(kind='hist', bins=20, title='ilość_sypialni')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['ilość_łazienek'].plot(kind='hist', bins=20, title='ilość_łazienek')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['rok_budowy'].plot(kind='hist', bins=20, title='rok_budowy')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['dzielnica'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='ilość_sypialni', y='dzielnica', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['dzielnica'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='ilość_łazienek', y='dzielnica', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_16['dzielnica'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_16, x='rok_budowy', y='dzielnica', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
### TEST
import random

_a = 0.3
_b = 0.5

f = lambda x: _a * x + _b # ground truth
g = lambda x: f(x) + random.gauss(0, 0.02) # a noisy version of f

n = 50 # number of examples

xs = [random.random() for _ in range(n)] # features
ys = list(map(g, xs)) # targets

ts = list(map(f, xs)) # we don't get to see this

xs = np.array(xs)
ys = np.array(ys)

import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(x=xs, y=ys)
fig.show()

Now, let's implement a standard linear regression model.

In [32]:
##########################################################
# TODO: Implement linear regression and compute its msle #
##########################################################
# features = ['m2', 'ilość_sypialni', 'ilość_łazienek', 'rok_budowy', 'parking_podziemny']
features = ['m2', 'ilość_sypialni', 'rok_budowy']
X = mieszkania[features].values
y = mieszkania['cena'].values

np.random.seed(2137)
#theta = np.random.randn(X.shape[1] + 1)
theta = np.full(X.shape[1], np.float64(0))
theta0 = np.float64(0)

X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

lr = 0.5
iterations = 1000
m = len(y)

def mse(ys, ps):
    # print(ps)
    return np.mean((ys - ps) * (ys - ps))

def msle(ys, ps):
    return np.mean((np.log1p(ys) - np.log1p(ps)) ** 2)

def evaluate(X, y, theta, theta0):
    return msle(y, X @ theta + theta0)

# def eval(a, b, x, yy):
#     return mse(a * x + b, yy)

def eval(X, theta, theta0):
    return (X @ theta + theta0)

a, b = np.float64(0), np.float64(0)

def linear_regression(X, y, theta, theta0):
    losses = []
    # Xx = X[:, 0]
    # Xx = X
    a = 0.0
    b = 0.0
    for i in range(iterations):
        f = y - (X @ theta + theta0)
        theta -= lr * (-2 * X.T @ f / m)
        theta0 -= lr * (-2 * f.sum() / m)

        # fa = y - (a * Xx + b)
        # a -= lr * (-2 * Xx.dot(fa).sum() / len(Xx))
        # b -= lr * (-2 * fa.sum() / len(Xx))

        # loss = eval(a, b, Xx, y)
        loss = mse(eval(X, theta, theta0), y)
        losses.append(loss)

    return theta

# linear_regression(X, y, theta)
# import plotly.graph_objects as go
# import plotly.express as px

# fig = px.scatter(x=X[:, 0], y=y)
# fig.show()

theta = linear_regression(X, y, theta, theta0)
print(theta)

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.LinearRegression()
regr.fit(X, y)
print(regr.coef_)
predy = regr.predict(X)
print(theta0)
print(mse(eval(X, theta, theta0), y))
# print(b)
# print(predy[:10])
# print((a * X[:, 0] + b)[:10])
# print(mean_squared_error(X, predy))
# print(mse(a * X[:, 0] + b, y))

# import matplotlib.pyplot as plt

# # # Plot outputs
# plt.scatter(X, y, color="black")
# plt.plot(X, eval(X, theta, theta0), color="blue", linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

# a1 = np.array([1.0, 1.0, 2.0, 2.0, 4.0])
# a2 = np.array([0.6, 1.29, 1.99, 2.69, 3.4])
# mse(a1, a2)

[225224.67050134  15926.16760201  19686.74949262]
[225224.67050859  15926.16760928  19686.7494949 ]
0.0
272358940053.01648


Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint:
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [ ]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code.

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint:
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2:
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
###############################################
# TODO: Implement the feature engineering part #
###############################################

In [ ]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################